In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# import the libraries :
from keras.layers import *
from keras.models import *
from numpy import asarray
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# handling the warning :
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn

In [3]:
# read the dataset and find the train, test set size : ws = 2

def read_data(file_path):
    data = pd.read_csv(file_path)
    
    # drop all the infinite values :
    data = data.dropna()
    data = data.values
    #data = get_the_modified_dataset(file_path,ws,data)
    n = data.shape[1] - 1
    x = data[:,0:n]
    y = data[:,n]

    return x,y

file_path = './gdrive/My Drive/MSC Thesis ROOT FOLDER/MSC_THESIS_NEW_START/prev_features_train_9.csv'
X_train,y_train = read_data(file_path)
file_path = './gdrive/My Drive/MSC Thesis ROOT FOLDER/MSC_THESIS_NEW_START/prev_features_test_9.csv'
X_test,y_test = read_data(file_path)

# perform normalization :
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print('Train shape : ',X_train.shape,y_train.shape)
print('Test shape : ',X_test.shape,y_test.shape)

Train shape :  (27001, 75) (27001,)
Test shape :  (14090, 75) (14090,)


In [4]:
print('without undersampling : ')
print('Train residue statistics : ')
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )

print('Test residue statistics : ')
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )


without undersampling : 
Train residue statistics : 
positive class :  1013
negative class :  25988
positive ratio : negative ratio  3.8979529013390795  :  100.0
Test residue statistics : 
positive class :  507
negative class :  13583
positive ratio : negative ratio  3.7326069351395126  :  100.0


In [5]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42,replacement=True)
rus_X_train,rus_y_train = rus.fit_resample(X_train,y_train)

print('with undersampling : ')
print('Train residue statistics : ')
unique_elements, counts_elements = np.unique(rus_y_train, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )

print('Test residue statistics : ')
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print( 'positive class : ' , np.asarray((counts_elements)[1]) )
print( 'negative class : ' , np.asarray((counts_elements)[0]) )
print('positive ratio : negative ratio ' , str( ( (counts_elements)[1] / (counts_elements)[0] ) * 100 ) , ' : ', str( ( (counts_elements)[0] / (counts_elements)[0] ) * 100 ) )



with undersampling : 
Train residue statistics : 
positive class :  1013
negative class :  1013
positive ratio : negative ratio  100.0  :  100.0
Test residue statistics : 
positive class :  507
negative class :  13583
positive ratio : negative ratio  3.7326069351395126  :  100.0


In [14]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [15]:
# import libraries for CNN :

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


import numpy
from tensorflow import keras
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import initializers

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils


In [16]:
# make the residue into image :
ws = 4
def residue_to_image(ws,X):
    
    start_index = ws ;
    end_index =  len(X) - 1  - ws ;
    
    print('start index : ',start_index)
    print('end index : ',end_index)

    X_images = np.zeros((1,X.shape[1]),dtype=float)
     
    
    for i in range(start_index,end_index+1):
        temp_X = X[i-ws:i+ws+1,:]
          
     
        X_images = np.concatenate( (X_images,temp_X),axis=0 )
        
    
    return X_images          

print('Train information : ')
X_images = residue_to_image(ws,rus_X_train)
X_train_images = np.delete(X_images,1,axis=0)
print('X_train image : ',X_train_images.shape) 

print('Test information : ')
X_images = residue_to_image(ws,X_test)
X_test_images = np.delete(X_images,1,axis=0)
print('X_test image : ',X_test_images.shape)  

y_train_images = rus_y_train[ws : ( len(rus_X_train) - 1 - ws ) + 1]
print('y_train image ',y_train_images.shape) 

y_test_images = y_test[ws : ( len(y_test) - 1 - ws ) + 1]
print('y_test image ',y_test_images.shape) 


X_train_images = X_train_images.reshape(rus_X_train.shape[0] - ws*2,2*ws+1,rus_X_train.shape[1])
X_test_images = X_test_images.reshape(X_test.shape[0] - ws*2,2*ws+1,X_test.shape[1])

print('My Dataset dataset : ')
print('My Dataset - X_train : ',X_train_images.shape)
print('My Dataset - y_train : ',y_train_images.shape)
print('My Dataset - X_test : ',X_test_images.shape)
print('My Dataset - y_test : ',y_test_images.shape)


# Reshape the image so that it can work with the keras API :
X_train_images = X_train_images.reshape(X_train_images.shape[0],2*ws+1,X_train_images.shape[2],1)
X_test_images = X_test_images.reshape(X_test_images.shape[0],2*ws+1,X_test_images.shape[2],1)

print('X_train_images : ',X_train_images.shape)
print('X_test_images : ',X_test_images.shape)

Train information : 
start index :  4
end index :  2021
X_train image :  (18162, 75)
Test information : 
start index :  4
end index :  14085
X_test image :  (126738, 75)
y_train image  (2018,)
y_test image  (14082,)
My Dataset dataset : 
My Dataset - X_train :  (2018, 9, 75)
My Dataset - y_train :  (2018,)
My Dataset - X_test :  (14082, 9, 75)
My Dataset - y_test :  (14082,)
X_train_images :  (2018, 9, 75, 1)
X_test_images :  (14082, 9, 75, 1)


In [17]:
# Parameter search space
no_of_filters = list(range(12,257,15)) ; 

kernel_size_1 = list(range(2,5));
kernel_size_2 = list(range(2,5));
pool_size = list(range(2,4));

units_1 = list(range(12,257,15)) ;
rate_1 = [0.3,0.5,0.7,0.9];

units_2 = list(range(12,257,15)) ;
rate_2 = [0.3,0.5,0.7,0.9];

learning_rate = [1e-2, 1e-3];


print('no_of_filters : ',no_of_filters)
print('kernel size - 1 : ', kernel_size_1)
print('kernel size - 2 : ', kernel_size_2)
print('pool size - 1 : ', pool_size)
print('units - 1 : ', units_1)
print('rate - 1 : ',rate_1)
print('units - 2 : ', units_2)
print('rate - 2 : ',rate_2)
print('learning rate - 1 : ',learning_rate)

no_of_filters :  [12, 27, 42, 57, 72, 87, 102, 117, 132, 147, 162, 177, 192, 207, 222, 237, 252]
kernel size - 1 :  [2, 3, 4]
kernel size - 2 :  [2, 3, 4]
pool size - 1 :  [2, 3]
units - 1 :  [12, 27, 42, 57, 72, 87, 102, 117, 132, 147, 162, 177, 192, 207, 222, 237, 252]
rate - 1 :  [0.3, 0.5, 0.7, 0.9]
units - 2 :  [12, 27, 42, 57, 72, 87, 102, 117, 132, 147, 162, 177, 192, 207, 222, 237, 252]
rate - 2 :  [0.3, 0.5, 0.7, 0.9]
learning rate - 1 :  [0.01, 0.001]


In [18]:
from sklearn.metrics import confusion_matrix

import math


def objective(solution):
    #print(solution)
    
    model = Sequential()
   
    model.add( Conv2D(filters=solution[0],kernel_size=solution[1],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu',
            input_shape=(2*ws+1,X_train_images.shape[2],1) ) )
    model.add( Conv2D(filters=solution[0],kernel_size=solution[2],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu') ) 
    model.add( MaxPool2D( pool_size=solution[3]) )
    model.add(Flatten())
    model.add(Dense(units=solution[4],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=solution[5]))

    model.add(Dense(units=solution[6],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=solution[7]))
    
    model.add(Dense(1, activation='sigmoid'))
    #model.summary()
    
    
    model.compile(optimizer=keras.optimizers.Adam(solution[8]),loss='binary_crossentropy', metrics='accuracy' )
    

    model.fit(X_train_images, y_train_images, batch_size=120, epochs=10, validation_data=(X_test_images, y_test_images))
    y_pred = np.round(    model.predict(X_test_images), 0  )
    tn, fp, fn, tp = confusion_matrix(y_test_images, y_pred).ravel()
    stat = [tn,fp,fn,tp]

    sen = tp / (tp + fn)
    # f1-score :
    pre = tp/(tp+fp)
    fitness = (2*pre*sen) / (pre + sen)
    print('f1-score : ',fitness )

    if( math.isnan(fitness)):
       print('fitness : ',fitness)
       return 0,0;
    else:
       print('fitness : ',fitness)   
       return fitness,stat;


In [19]:
import random
from math import exp

global_stat = []

def simulated_annealing(bounds,n_iterations,temp):
    
    global no_of_filters,kernel_size_1,kernel_size_2,pool_size,units_1,rate_1,units_2,rate_2,learning_rate;

    # generate an initial point  
    no_of_filters_temp = random.choice(no_of_filters) ;

    kernel_size_1_temp = random.choice(kernel_size_1);
    kernel_size_2_temp = random.choice(kernel_size_2);
    
    pool_size_temp = random.choice(pool_size);

    units_1_temp = random.choice( units_1 ) ;
    rate_1_temp = random.choice( rate_1 );
   
    units_2_temp = random.choice( units_2 ) ;
    rate_2_temp = random.choice( rate_2 );
    
    learning_rate_temp = random.choice( learning_rate ) ;
    # Parameters of the best solution : [12, 4, 4, 2, 252, 0.3, 252, 0.3, 0.01]
    best = [no_of_filters_temp, kernel_size_1_temp,kernel_size_2_temp,pool_size_temp,units_1_temp,rate_1_temp,units_2_temp,rate_2_temp,learning_rate_temp] ;
    #print(best)

    # evaluate the initial point :
    best_eval,stat = objective(best) ;

    
    # current working solution :
    curr,curr_eval = best,best_eval

    
    for i in range(0,n_iterations):
        
        # take a step :
        rand_index_1 = random.randint(0,len(curr)-1);
        rand_index_2 = random.randint(0,len(curr)-1);
        rand_index_3 = random.randint(0,len(curr)-1);

        candidate = curr.copy() ;  
        candidate[rand_index_1] = random.choice( bounds[rand_index_1] );
        candidate[rand_index_2] = random.choice( bounds[rand_index_2] );
        candidate[rand_index_3] = random.choice( bounds[rand_index_3] );
        
        # evaluate candidate point :
        candidate_eval,stat = objective(candidate);

        # check for new best solution :
        if candidate_eval > best_eval :
           #store new best point :
           best,best_eval = candidate,candidate_eval ;
           global global_stat
           global_stat = stat.copy()

        # report progress :
        print('iter no : and best evaluation : ',i,best_eval);

        # diff between candidate and current point evaluation :
        diff = candidate_eval - curr_eval ;

        #calculate temperature for current eopch :
        t = temp / float(i+1) ;

        # calculate metropolis acceptance criterion :
        metropolis = exp(-diff/t)

        # check we should keep the new point :
        if diff<0 or random.random()<metropolis:
           #store the new current point :
           curr,curr_eval = candidate, candidate_eval ;

        print('iter no : ',i,' is completed')
    
    return [best,best_eval,global_stat] ;   
   
    

In [20]:
'''
print('no_of_filters : ',no_of_filters)
print('kernel size - 1 : ', kernel_size_1)
print('kernel size - 2 : ', kernel_size_2)
print('pool size - 1 : ', pool_size)
print('units - 1 : ', units_1)
print('rate - 1 : ',rate_1)
print('units - 2 : ', units_2)
print('rate - 2 : ',rate_2)
print('learning rate - 1 : ',learning_rate)
'''
bounds = [ [ min(no_of_filters),max(no_of_filters)],
          [ min(kernel_size_1),max(kernel_size_1)],
          [ min(kernel_size_2),max(kernel_size_2)],
          [ min(pool_size),max(pool_size)],
          [ min(units_1),max(units_1)],
          [ min(rate_1),max(rate_1)],
          [ min(units_2),max(units_2)],
          [ min(rate_2),max(rate_2)],
          [ min(learning_rate),max(learning_rate)]
         ] ;
print(bounds)


iterations = 10 ;
temp = 15 ;

import time
start_time = time.time()
best,best_eval,global_stat = simulated_annealing(bounds,iterations,temp) ;
end_time = time.time()

print('Time required : ',(end_time-start_time)/60 )
print('the best parameter set : ',best )
print('the best fitness value : ',best_eval )
print('tn,fp,fn,tp : ' , global_stat )
print('done')



[[12, 252], [2, 4], [2, 4], [2, 3], [12, 252], [0.3, 0.9], [12, 252], [0.3, 0.9], [0.001, 0.01]]
Epoch 1/10
17/17 [==============================] - 4s 169ms/step - loss: 0.6934 - accuracy: 0.4999 - val_loss: 0.5780 - val_accuracy: 0.7228
Epoch 2/10
17/17 [==============================] - 2s 141ms/step - loss: 0.4559 - accuracy: 0.8117 - val_loss: 0.4923 - val_accuracy: 0.8286
Epoch 3/10
17/17 [==============================] - 2s 144ms/step - loss: 0.2885 - accuracy: 0.8830 - val_loss: 0.4954 - val_accuracy: 0.8406
Epoch 4/10
17/17 [==============================] - 2s 145ms/step - loss: 0.2242 - accuracy: 0.9138 - val_loss: 1.1799 - val_accuracy: 0.7079
Epoch 5/10
17/17 [==============================] - 2s 143ms/step - loss: 0.1845 - accuracy: 0.9297 - val_loss: 1.1891 - val_accuracy: 0.6733
Epoch 6/10
17/17 [==============================] - 2s 142ms/step - loss: 0.1202 - accuracy: 0.9499 - val_loss: 1.7499 - val_accuracy: 0.6301
Epoch 7/10
 1/17 [>.............................] -

KeyboardInterrupt: ignored

In [13]:
print(global_stat)
# tn,fp,fn,tp
tn = global_stat[0] ; fp = global_stat[1] ; fn = global_stat[2] ; tp = global_stat[3];

# accuracy :
acc = (tp + tn) / (tp + fp + fn + tn)
print('Accuracy : ',"{:.2f}".format(acc) )

# sensitivity :
sen = tp / (tp + fn)
print('Sensitivity : ',"{:.4f}".format(sen) )

# specificity :
spec = tn / (fp + tn)
print('Specificity : ',"{:.4f}".format(spec) )

# mcc :
import math
mcc = (tp * tn - fp * fn) / math.sqrt( (tp+fp) * (fn+tn) * (fp+tn) * (tp+fn) )
print('MCC : ',"{:.4f}".format(mcc) ) 

# f1-score :
# recall -> sen
pre = tp/(tp+fp)
f1_score = (2*pre*sen) / (pre + sen)
print('f1-score : ',"{:.4f}".format(f1_score) )


 

[5961, 7614, 108, 399]
Accuracy :  0.45
Sensitivity :  0.7870
Specificity :  0.4391
MCC :  0.0851
f1-score :  0.0937


In [ ]:
from sklearn.metrics import confusion_matrix

import math

global_stat = []

def objective(solution):
    #print(solution)
    
    model = Sequential()
   
    '''
    
    no_of_filters -> 0
    
    kernel_size_1 -> 1
    kernel_size_2 -> 2
    
    pool_size -> 3
    
    units_1 -> 4
    rate_1 -> 5
    
    units_2 -> 6
    rate_2 -> 7
    
    learning_rate -> 8
    
    '''

    model.add( Conv2D(filters=solution[0],kernel_size=solution[1],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu',
            input_shape=(2*ws+1,X_train_images.shape[2],1) ) )
    model.add( Conv2D(filters=solution[0],kernel_size=solution[2],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu') ) 
    model.add( MaxPool2D( pool_size=solution[3]) )
    model.add(Flatten())
    model.add(Dense(units=solution[4],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=solution[5]))

    model.add(Dense(units=solution[6],kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
    model.add(Dropout(rate=solution[7]))
    
    model.add(Dense(1, activation='sigmoid'))
    #model.summary()
    
    
    model.compile(optimizer=keras.optimizers.Adam(solution[8]),loss='binary_crossentropy', metrics='accuracy' )
    

    model.fit(X_train_images, y_train_images, batch_size=120, epochs=10, validation_data=(X_test_images, y_test_images))
    y_pred = np.round(    model.predict(X_test_images), 0  )
    tn, fp, fn, tp = confusion_matrix(y_test_images, y_pred).ravel()
    stat = [tn,fp,fn,tp]

    sen = tp / (tp + fn)
    # f1-score :
    pre = tp/(tp+fp)
    fitness = (2*pre*sen) / (pre + sen)
    print('f1-score : ',fitness )

    if( math.isnan(fitness)):
       print('fitness : ',fitness)
       return 0;
    else:
       print('fitness : ',fitness)   
       return fitness;


In [ ]:
import random
from math import exp

def simulated_annealing(bounds,n_iterations,temp):
    
    global no_of_filters,kernel_size_1,kernel_size_2,pool_size,units_1,rate_1,units_2,rate_2,learning_rate;

    # generate an initial point  
    no_of_filters_temp = random.choice(no_of_filters) ;

    kernel_size_1_temp = random.choice(kernel_size_1);
    kernel_size_2_temp = random.choice(kernel_size_2);
    
    pool_size_temp = random.choice(pool_size);

    units_1_temp = random.choice( units_1 ) ;
    rate_1_temp = random.choice( rate_1 );
   
    units_2_temp = random.choice( units_2 ) ;
    rate_2_temp = random.choice( rate_2 );
    
    learning_rate_temp = random.choice( learning_rate ) ;
    # Parameters of the best solution : [12, 4, 4, 2, 252, 0.3, 252, 0.3, 0.01]
    best = [no_of_filters_temp, kernel_size_1_temp,kernel_size_2_temp,pool_size_temp,units_1_temp,rate_1_temp,units_2_temp,rate_2_temp,learning_rate_temp] ;
    #print(best)

    # evaluate the initial point :
    best_eval = objective(best) ;

    
    # current working solution :
    curr,curr_eval = best,best_eval

    
    for i in range(0,n_iterations):
        
        # take a step :
        rand_index_1 = random.randint(0,len(curr)-1);
        rand_index_2 = random.randint(0,len(curr)-1);
        rand_index_3 = random.randint(0,len(curr)-1);

        candidate = curr.copy() ;  
        candidate[rand_index_1] = random.choice( bounds[rand_index_1] );
        candidate[rand_index_2] = random.choice( bounds[rand_index_2] );
        candidate[rand_index_3] = random.choice( bounds[rand_index_3] );
        
        # evaluate candidate point :
        candidate_eval = objective(candidate);

        # check for new best solution :
        if candidate_eval > best_eval :
           #store new best point :
           best,best_eval = candidate,candidate_eval ;

        # report progress :
        print('iter no : and best evaluation : ',i,best_eval);

        # diff between candidate and current point evaluation :
        diff = candidate_eval - curr_eval ;

        #calculate temperature for current eopch :
        t = temp / float(i+1) ;

        # calculate metropolis acceptance criterion :
        metropolis = exp(-diff/t)

        # check we should keep the new point :
        if diff<0 or random.random()<metropolis:
           #store the new current point :
           curr,curr_eval = candidate, candidate_eval ;

        print('iter no : ',i,' is completed')
    
    return [best,best_eval] ;   


    
    

In [ ]:
'''
print('no_of_filters : ',no_of_filters)
print('kernel size - 1 : ', kernel_size_1)
print('kernel size - 2 : ', kernel_size_2)
print('pool size - 1 : ', pool_size)
print('units - 1 : ', units_1)
print('rate - 1 : ',rate_1)
print('units - 2 : ', units_2)
print('rate - 2 : ',rate_2)
print('learning rate - 1 : ',learning_rate)
'''
bounds = [ [ min(no_of_filters),max(no_of_filters)],
          [ min(kernel_size_1),max(kernel_size_1)],
          [ min(kernel_size_2),max(kernel_size_2)],
          [ min(pool_size),max(pool_size)],
          [ min(units_1),max(units_1)],
          [ min(rate_1),max(rate_1)],
          [ min(units_2),max(units_2)],
          [ min(rate_2),max(rate_2)],
          [ min(learning_rate),max(learning_rate)]
         ] ;
print(bounds)


iterations = 10 ;
temp = 10 ;

import time
start_time = time.time()
best,best_eval = simulated_annealing(bounds,iterations,temp) ;
end_time = time.time()

print('Time required : ',(end_time-start_time)/60 )
print('the best parameter set : ',best )
print('the best fitness value : ',best_eval )
print('done')



[[12, 252], [2, 4], [2, 4], [2, 3], [12, 252], [0.3, 0.9], [12, 252], [0.3, 0.9], [0.001, 0.01]]
Epoch 1/10
17/17 [==============================] - 6s 302ms/step - loss: 0.7002 - accuracy: 0.5344 - val_loss: 0.8145 - val_accuracy: 0.3120
Epoch 2/10
17/17 [==============================] - 4s 275ms/step - loss: 0.5432 - accuracy: 0.7301 - val_loss: 0.7542 - val_accuracy: 0.7208
Epoch 3/10
17/17 [==============================] - 4s 262ms/step - loss: 0.2905 - accuracy: 0.8830 - val_loss: 1.2866 - val_accuracy: 0.5912
Epoch 4/10
17/17 [==============================] - 4s 271ms/step - loss: 0.1656 - accuracy: 0.9337 - val_loss: 1.5373 - val_accuracy: 0.6253
Epoch 5/10
17/17 [==============================] - 4s 265ms/step - loss: 0.1221 - accuracy: 0.9507 - val_loss: 1.8528 - val_accuracy: 0.6298
Epoch 6/10
17/17 [==============================] - 4s 271ms/step - loss: 0.1107 - accuracy: 0.9562 - val_loss: 1.6016 - val_accuracy: 0.6248
Epoch 7/10
17/17 [==============================] -

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in long_scalars


f1-score :  nan
fitness :  nan
iter no : and best evaluation :  7 0.09999999999999999
iter no :  7  is completed
Epoch 1/10
17/17 [==============================] - 255s 16s/step - loss: 1.4603 - accuracy: 0.4872 - val_loss: 0.7109 - val_accuracy: 0.0360
Epoch 2/10
17/17 [==============================] - 251s 15s/step - loss: 0.6955 - accuracy: 0.4816 - val_loss: 0.6716 - val_accuracy: 0.9571
Epoch 3/10
17/17 [==============================] - 251s 15s/step - loss: 0.6809 - accuracy: 0.5720 - val_loss: 0.7160 - val_accuracy: 0.9034
Epoch 4/10
17/17 [==============================] - 253s 15s/step - loss: 0.6161 - accuracy: 0.6590 - val_loss: 2.1710 - val_accuracy: 0.3943
Epoch 5/10
17/17 [==============================] - 255s 16s/step - loss: 0.4951 - accuracy: 0.7916 - val_loss: 2.1355 - val_accuracy: 0.3777
Epoch 6/10
17/17 [==============================] - 253s 16s/step - loss: 0.3416 - accuracy: 0.8576 - val_loss: 5.8338 - val_accuracy: 0.0939
Epoch 7/10
17/17 [=================

In [ ]:
print('the best parameter set : ',best )
print('the best fitness value : ',best_eval )


the best parameter set :  [12, 4, 4, 2, 252, 0.3, 252, 0.3, 0.01]
the best fitness value :  0.09999999999999999


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import numpy
from tensorflow import keras
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import initializers

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

import tensorflow.keras
from keras.utils.vis_utils import plot_model

'''

no_of_kernel,kernel_size_1,kernel_size_2,pool_kernel_size,size_each_dense_layer_1,dropout_rate_1,size_each_dense_layer_2,dropout_rate_2,learning_rate
Parameters of the best solution : [12, 4, 4, 2, 252, 0.3, 252, 0.3, 0.01]

'''

model = Sequential()
model.add( Conv2D(filters=12,kernel_size=4,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu',
            input_shape=(2*ws+1,X_train_images.shape[2],1) ) )
model.add( Conv2D(filters=12,kernel_size=4,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu') ) 
model.add( MaxPool2D( pool_size=2) )
model.add(Flatten())
model.add(Dense(units=252,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=252,kernel_initializer=initializers.RandomNormal(stddev=0.01), bias_initializer=initializers.Zeros(),activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(1, activation='sigmoid'))
    
model.summary()
#plot_model(model, to_file='model_plot.png') 
model.compile(optimizer=keras.optimizers.Adam(0.01),loss='binary_crossentropy', metrics='accuracy' )



Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 6, 72, 12)         204       
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 3, 69, 12)         2316      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 1, 34, 12)         0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 408)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 252)               103068    
_________________________________________________________________
dropout_50 (Dropout)         (None, 252)               0         
_________________________________________________________________
dense_76 (Dense)             (None, 252)             

In [ ]:
model.fit( X_train_images, y_train_images, epochs=10, validation_data=(X_test_images,y_test_images) )

Epoch 1/10
64/64 [==============================] - 5s 65ms/step - loss: 0.4630 - accuracy: 0.7576 - val_loss: 0.5915 - val_accuracy: 0.7256
Epoch 2/10
64/64 [==============================] - 3s 51ms/step - loss: 0.1963 - accuracy: 0.9328 - val_loss: 1.9875 - val_accuracy: 0.5817
Epoch 3/10
64/64 [==============================] - 4s 56ms/step - loss: 0.1674 - accuracy: 0.9360 - val_loss: 2.5286 - val_accuracy: 0.6331
Epoch 4/10
64/64 [==============================] - 4s 59ms/step - loss: 0.1330 - accuracy: 0.9539 - val_loss: 1.6779 - val_accuracy: 0.5827
Epoch 5/10
64/64 [==============================] - 3s 52ms/step - loss: 0.0794 - accuracy: 0.9727 - val_loss: 2.1390 - val_accuracy: 0.5552
Epoch 6/10
64/64 [==============================] - 3s 55ms/step - loss: 0.0720 - accuracy: 0.9813 - val_loss: 4.1670 - val_accuracy: 0.6050
Epoch 7/10
64/64 [==============================] - 3s 51ms/step - loss: 0.1231 - accuracy: 0.9644 - val_loss: 4.1200 - val_accuracy: 0.5317
Epoch 8/10
64

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = np.round(    model.predict(X_test_images), 0  )
print( confusion_matrix(y_test_images, y_pred ) )

[[5851 7724]
 [ 135  372]]


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test_images, y_pred).ravel()
print(tn, fp, fn, tp)

# accuracy :
acc = (tp + tn) / (tp + fp + fn + tn)
print('Accuracy : ',"{:.2f}".format(acc) )

# sensitivity :
sen = tp / (tp + fn)
print('Sensitivity : ',"{:.2f}".format(sen) )

# specificity :
spec = tn / (fp + tn)
print('Specificity : ',"{:.2f}".format(spec) )


# mcc :
import math
mcc = (tp * tn - fp * fn) / math.sqrt( (tp+fp) * (fn+tn) * (fp+tn) * (tp+fn) )
print('MCC : ',"{:.2f}".format(mcc) ) 

# f1-score :
# recall -> sen
pre = tp/(tp+fp)
f1_score = (2*pre*sen) / (pre + sen)
print('f1-score : ',"{:.2f}".format(f1_score) )


5851 7724 135 372
Accuracy :  0.44
Sensitivity :  0.73
Specificity :  0.43
MCC :  0.06
f1-score :  0.09
